# AMI pipeline

ERDA downloader

flat-bug

order classifier

species classifier

In [ ]:
import os
from inspect import getmembers_static
from os.path import join
from types import SimpleNamespace
import importlib

import cv2
import torch
from fastai.vision.all import load_learner, CategoryMap
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from torchvision import transforms
kavi_pipeline = importlib.import_module("009_kavi_pipeline_integration")

In [ ]:
learner_path = "/home/george/codes/lepinet/data/lepi/models/04-lepi-prod_model1-save-hierarchy"

In [ ]:
img_path = "/home/george/codes/lepinet/data/lepi/images/1730978/80e33d32259a9f4f034b2cb94f98ce8b5b70bfcd.jpeg"
img_dir = "/home/george/codes/lepinet/data/lepi/images/1730978"
kavi_img_path = "/home/george/codes/lepinet/data/kavi/20240708225609-snapshot.jpg"

In [ ]:
learn=load_learner(learner_path, cpu=False)

In [ ]:
learn.model

In [ ]:
len(learn.dls.vocab)

In [ ]:
type(learn.dls.valid_ds)

In [ ]:
type(learn.dls.loaders[1].after_item)

In [ ]:
getmembers_static(learn.dls.valid)

In [ ]:
getmembers_static(learn.dls.loaders[1])

In [ ]:
learn.dls.valid==learn.dls.loaders[1]

In [ ]:
learn.dls.valid.new()

In [ ]:
type(learn.dls.valid_ds)

In [ ]:
img=learn.dls.loaders[1].after_item(Image.open(img_path))

In [ ]:
getmembers_static(img)

In [ ]:
filenames = [join(img_dir, f) for f in os.listdir(img_dir)]

In [ ]:
type(learn.dls.valid_ds.tls)

In [ ]:
learn.dls.valid_ds.tls[0].dataloaders(img_path)

In [ ]:
getmembers_static(learn.dls.valid_ds)

In [ ]:
learn.dls.valid.after_item()

In [ ]:
test_dl = learn.dls.test_dl(filenames,)

In [ ]:
type(test_dl)

In [ ]:
getmembers_static(test_dl)

In [ ]:
imgs=[]
n_limit = 16
to_tensor = transforms.ToTensor()
for i, img_path in enumerate(os.listdir(img_dir)):
    if i == n_limit:
        break
    img=Image.open(join(img_dir, img_path))
    img=np.array(img)
    imgs.append(img)
len(imgs)

In [ ]:
def prepareCrop(imageCrop, img_size=128):
    image = cv2.resize(imageCrop, (img_size, img_size))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = Image.fromarray(image)
    image = transforms.ToTensor()(image) #.unsqueeze_(0)
    return image

In [ ]:
type(imgs[0])

In [ ]:
test_dl = learn.dls.test_dl(imgs)

In [ ]:
with learn.no_bar(), learn.no_logging():
    preds, _ = learn.get_preds(dl=test_dl)

In [ ]:
getmembers_static(learn.dls.loaders[1])

Test Kavi images

In [ ]:
classifier=kavi_pipeline.FastaiSpeciesClassifier(speciesModelPath="/home/george/codes/lepinet/data/lepi/models/04-lepi-prod_model1-save-hierarchy-id2name", output_type='all')

In [ ]:
image = Image.open(kavi_img_path)

In [ ]:
image = cv2.imread(kavi_img_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

In [ ]:
bboxes=[
    {"x1": 3240,"x2": 3392,"y1": 277, "y2": 433},
    {"x1": 3024,"x2": 3112,"y1": 839,"y2": 983},
    {"x1": 816,"x2": 921,"y1": 170,"y2": 343}
]
detections = []
for i,b in enumerate(bboxes):
    bbox = SimpleNamespace(**b)
    detection = SimpleNamespace(id=f'{i+1}', bbox=bbox)
    detections.append(detection)

batch = classifier.batchFromDetections(image, detections)
results = classifier.classifySpeciesBatch(batch)

# Print results
for i, res in enumerate(results):
    print("\n--- Detection Result ---")
    print("bboxes:", bboxes[i])
    print("Detection ID:", res["id"])
    print("Predicted Labels (Hierarchy):", res["label"])
    print("Predicted Labels ID (Hierarchy):", res["labelId"])
    print("Confidence Scores:", res["confidence_value"])
    
    # plt.imshow(batch["imagesInBatch"][i])
    # plt.show()

In [ ]:
for b in batch["imagesInBatch"]:
    plt.imshow(b)
    plt.show()

In [ ]:
# plt.figsize()
plt.imshow(image)

Deprecated code:

In [ ]:
    # Create a custom test dataset
    class PILImagesList(L):
        def __getitem__(self, idx):
            return pil_images[idx]
    
    # Create a custom test dataset class that directly accepts PIL images
    class PILImagesDataset(Dataset):
        def __init__(self, images):
            self.images = images
            
        def __len__(self):
            return len(self.images)
            
        def __getitem__(self, idx):
            # Return the PIL image and a dummy label (0)
            return PILItem(self.images[idx], None, 0)

In [ ]:
def gen_level_idx(vocab, hierarchy):
    """
    Returns a list of integers of the size of vocab indicating the hierarchical level of the taxa at index i.
    - Species is level 0, Genus 1, Family 2, etc.
    - Missing values are noted with -1.

    Args:
    - vocab (list): List of taxa names to find levels for.
    - hierarchy (dict): Nested dictionary representing taxonomic hierarchy.

    Returns:
    - np.ndarray: Array of level indices for each taxa in vocab.
    """
    level_lookup = {}

    def traverse(node, level=0):
        """Recursively traverse the hierarchy and store levels."""
        for key, subnode in node.items():
            level_lookup[key] = level  # Assign level to the taxon
            if isinstance(subnode, dict):
                traverse(subnode, level + 1)
            elif isinstance(subnode, list):  # Leaf nodes (species level)
                for species in subnode:
                    level_lookup[species] = level + 1

    # Build the level lookup dictionary
    traverse(hierarchy)  # Start from -1 so species end up at level 0

    # Assign levels to vocab, default to -1 if missing
    indices = np.array([level_lookup.get(v, -1) for v in vocab], dtype=int)

    # Invert the indices, so species is 0, genus is 1 etc
    # indices = np.where(indices < 0, indices, indices.max()-indices)

    # Warning for missing values
    missing_count = np.sum(indices == -1)
    if missing_count > 0:
        print(f"[Warning] Missing values in taxa dictionary: {missing_count}.")

    return indices

def split_preds(preds:torch.Tensor, indices:np.ndarray):
    """Returns split preds using indices.

    `preds` is a batch of predictions.
    """
    out_preds = []
    indices = torch.from_numpy(indices)
    for i in range(indices.max()+1):
        out_preds += [preds[:,indices==i].cpu().numpy()]
    return out_preds

def get_pred_conf(preds:torch.Tensor, vocab:CategoryMap, indices:np.ndarray):
    """Returns predicted labels and confidence for each pred and for each 
    hierarchy level.

    `preds` is a batch of predictions.
    """
    out_preds = []
    out_confs = []
    indices = torch.from_numpy(indices)
    for i in range(indices.max()+1):
        one_level_pred = preds[:,indices==i].cpu().numpy()
        one_level_prd = vocab[indices==i][one_level_pred.argmax(axis=1)]
        one_level_cnf = one_level_pred.max(axis=1)
        out_preds += [one_level_prd]
        out_confs += [one_level_cnf]
    return np.array(out_preds).swapaxes(0,1), np.array(out_confs).swapaxes(0,1)


indices=gen_level_idx(learn.dls.vocab, learn.hierarchy)
prds, cnfs = get_pred_conf(preds, learn.dls.vocab, indices)

In [ ]:
prds, cnfs

In [ ]:
class AsyncSFTPParams(TypedDict):
    host: str
    port: int
    username: str
    password: str
    client_keys: list[str]

In [ ]:
sftp_params=AsyncSFTPParams(
        host="io.erda.au.dk",
        port=2222,
        username="gmo@ecos.au.dk",
        client_keys=["~/.ssh/id_rsa"])

remotepaths = "AMI/storage/mambo/2024/uva/NL1/2024_07_18"
localpath = "/home/george/codes/lepinet/data/mambo/images"

async with asyncssh.connect(**sftp_params) as conn:
    async with conn.start_sftp_client() as sftp:
        sftp.get(remotepaths, localpath)

In [ ]:
class AMIPipeline:
    def __init__(
        self,
        remote_dir: str,
        local_dir: str,
        max_queue_size: int = 16
    ):
        os.makedirs(local_dir, exist_ok=True)

        # Queues for managing pipeline stages
        self.download_queue = asyncio.Queue(maxsize=max_queue_size)

        